In [1]:
import pandas as pd # for easy and effective catalogue manipulation
import requests # for dealing with API
import json
from ibm_watson import AssistantV2
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
import visJS2jupyter.visJS_module
import networkx as nx
import matplotlib.pyplot as plt

In [3]:
! rm x5gon_catelogue.tsv*
! mkdir datasets
! wget https://gitlab.univ-nantes.fr/x5gon/x5gon-hackathon-datasets/raw/master/datasets/x5gon_catelogue.tsv
! mv x5gon_catelogue.tsv datasets/catalogue.tsv

rm: cannot remove 'x5gon_catelogue.tsv*': No such file or directory
--2020-02-25 10:35:38--  https://gitlab.univ-nantes.fr/x5gon/x5gon-hackathon-datasets/raw/master/datasets/x5gon_catelogue.tsv
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving gitlab.univ-nantes.fr (gitlab.univ-nantes.fr)... 193.52.101.66
Connecting to gitlab.univ-nantes.fr (gitlab.univ-nantes.fr)|193.52.101.66|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63850145 (61M) [text/plain]
Saving to: ‘x5gon_catelogue.tsv’

x5gon_catelogue.tsv 100%[===================>]  60.89M  1.16MB/s    in 41s     

2020-02-25 10:36:20 (1.50 MB/s) - ‘x5gon_catelogue.tsv’ saved [63850145/63850145]



In [21]:
! pip install --upgrade "ibm-watson>=4.2.1"

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 360 kB 2.4 MB/s eta 0:00:01
  Created wheel for ibm-watson: filename=ibm_watson-4.3.0-py3-none-any.whl size=354168 sha256=689e79ee4804122726fa83b53111e801345f3ce5d00bcbf8eaa06f51ce93561a
  Stored in directory: /home/odrec/.cache/pip/wheels/2c/e7/02/3ed9f7fcbaf5fbd036d80ca4eaf27721621ee4524450cadd13
  Created wheel for ibm-cloud-sdk-core: filename=ibm_cloud_sdk_core-1.5.1-py3-none-any.whl size=44488 sha256=54ac6e2d698c1ac056c01b2a04f7e6bf9fc25698062ccad26ec921b18cdfd95d
  Stored in directory: /home/odrec/.cache/pip/wheels/51/37/f5/0532ca77cb49d7405e309bb16e3bf3fd4488baabf0f326c63d
Successfully built ibm-watson ibm-cloud-sdk-core
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [93]:
! pip install visJS2jupyter
! pip install python-louvain
! pip install py_d3

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Created wheel for py-d3: filename=py_d3-0.3.0-py3-none-any.whl size=4330 sha256=6101d88dfe3e7b6c4cecc59741dc3bc812cdb085bcdb618b29297c7dd8c25a2c
  Stored in directory: /home/odrec/.cache/pip/wheels/dd/2a/20/2252a7a30ce76461b8dc7af5a7537b3c32e3cc7247a54c92da
Successfully built py-d3


In [2]:
list_parser = lambda x: x[1:-1].split(',')
catalogue = pd.read_csv("datasets/catalogue.tsv",
                        sep="\t",
                        converters={'keywords': list_parser,
                                    "concepts": list_parser})
# This is added in case initial dataset hasn't the right columns names:
catalogue.columns = ['id', 'title', 'language', 'type', 'keywords', 'concepts']
catalogue.set_index("id", inplace=True)

In [99]:
! wget https://gitlab.univ-nantes.fr/x5gon/x5gon-hackathon-datasets/raw/master/datasets/x5gon_user_data.psv
! mv x5gon_user_data.psv datasets/user_data.psv

--2020-02-25 15:37:19--  https://gitlab.univ-nantes.fr/x5gon/x5gon-hackathon-datasets/raw/master/datasets/x5gon_user_data.psv
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving gitlab.univ-nantes.fr (gitlab.univ-nantes.fr)... 193.52.101.66
Connecting to gitlab.univ-nantes.fr (gitlab.univ-nantes.fr)|193.52.101.66|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 255593745 (244M) [text/plain]
Saving to: ‘x5gon_user_data.psv’

x5gon_user_data.psv 100%[===================>] 243.75M  1.67MB/s    in 3m 3s   

2020-02-25 15:40:26 (1.33 MB/s) - ‘x5gon_user_data.psv’ saved [255593745/255593745]



In [4]:
authenticator = IAMAuthenticator('1_EAWNCSIlEhv8ipZPaMhu1_mKLC2KZL6WrUoAwEYj4T')
assistant = AssistantV2(
    version='2019-02-28',
    authenticator=authenticator
)

assistant.set_service_url('https://api.eu-de.assistant.watson.cloud.ibm.com/instances/65c8de3a-d320-4c22-9d5f-9d4fbcc4a415')

In [ ]:
response = assistant.create_session(
    assistant_id='97ca7bbc-4958-416a-945f-2896292b368c'
).get_result()


sess_id = response['session_id']

PLATFORM_URL = "https://platform.x5gon.org/api/v1/"
function = "search/"
parameter = "text="
query = ""

# The X5GON API is available at: 
PLATFORM_LAM_URL = "http://wp3.x5gon.org/"
HEADERS = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}
rid = 44900

list_parser = lambda x: x[1:-1].split("|")
users = pd.read_csv(
    "datasets/user_data.psv",
    sep="|",
    converters={"keywords": list_parser, "concepts": list_parser},
)
# This is added in case initial dataset hasn't the right columns names:
users.columns = ["id", "url", "timestamp"]
users.set_index("timestamp", inplace=True)
users.head(20)

response = assistant.message(
    assistant_id='97ca7bbc-4958-416a-945f-2896292b368c',
    session_id=sess_id,
    input={
        'message_type': 'text',
        'text': query
    }).get_result()
print(response['output']['generic'][0]['text'])

print("Ali:")
query = input()
flag = False
while True:

    response = assistant.message(
        assistant_id='97ca7bbc-4958-416a-945f-2896292b368c',
        session_id=sess_id,
        input={
            'message_type': 'text',
            'text': query
        }
    ).get_result()
    
    
    if response['output']['entities']:
        entities = response['output']['entities']
        entity = entities[0]['entity']
        
    if response['output']['intents']:
        intents = response['output']['intents']
        intent = intents[0]['intent']
    
    if intent == 'positive_sure':
        keyword = entities[0]['value']

        PLATFORM_URL = "https://platform.x5gon.org/api/v1/"
        function = "search/"
        parameter = "text="
        keyword = "perceptron"
        response = requests.get(PLATFORM_URL + function + "?" + parameter + keyword)
        r_json = response.json()

        # store query results in a dict of dicts {material_d: {material_data}}
        # also remove duplicates by comparing material descriptions
        materials = {}
        for result in r_json["rec_materials"]:
            materials[result["material_id"]] = result
            materials[result["material_id"]].pop("material_id", None)

        df_materials = pd.DataFrame(materials)
        df_materials = df_materials.T
        df_materials
        flag = True
    if not flag:
        print("\nCommunicator:")
        print(response['output']['generic'][0]['text'])
        if response['output']['entities']:
            entities = response['output']['entities']
            entity = entities[0]['entity']

        titles = []
        titles_k = []
      
        print("\nAli:")
        query = input()
    else: break
    
for key in df_materials.index:
    res = users[users.url == key].id
    if not res.empty:
        #print(f"The material {key} - {materials[key]['title']} was accessed by:")
        users_list = list(users[users.url == key].id.values)
        #print(users[users.url == key].id.values)
        #print("")


#Due to problems with IBM Watson Assistant V2 API where no context is shown in the response, it's difficult 
#to send user-defined info back and forth. For now the dialog from here is hard-coded
print("Communicator:")
print("The users", users_list[0], users_list[1], users_list[2])
print("Have similar interests to you and would you like me to connect \
you to them so that you can exchange knowledge and maybe create a new learning community around the subject?")
print("Ali:")
query = input()
    


Hello, nice to meet you. How are you?
Ali:
I'm fine :)

Communicator:
Great! How can I help you?

Ali:
I want to learn about neural networks

Communicator:
I see you are interested in neural networks?

Ali:
thinking about it better, perceptron could be better for starters

Communicator:
Do I see correctly that you are interested in perceptron?

Ali:
